In [0]:
# Importing PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Fetching data from the Databricks table
df_spark = spark.sql("SELECT * FROM workspace.default.telco_customer_churn")


# Converting the PySpark DataFrame to a Pandas DataFrame
df = df_spark.toPandas()

# Displaying the first 10 rows of the Pandas DataFrame
df.head(10)


In [0]:
import pandas as pd
# عرض المعلومات حول الأعمدة (عدد القيم غير المفقودة ونوع البيانات)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors='coerce')
print(df.info())

# عرض الإحصاءات الأساسية مثل المتوسط والانحراف المعياري للبيانات
print(df.describe())


In [0]:
print(df.isnull().sum())

In [0]:
import matplotlib.pyplot as plt

df.hist(bins=50,figsize=(10,8))
plt.show()

In [0]:
import seaborn as sns

# حساب مصفوفة الارتباط بين الأعمدة
correlation_matrix = df.corr()

# رسم مصفوفة الارتباط
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()


In [0]:
sns.boxplot(x=df['TotalCharges'])
plt.show()

In [0]:
print(df["Churn"].value_counts())

sns.countplot(x='Churn',data = df)
plt.show()

In [0]:
import pandas as pd

# DATA 
print(df.isnull().sum())
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())
print(df.isnull().sum())

In [0]:
# تحويل الأعمدة الفئوية إلى متغيرات رقمية باستخدام One-Hot Encoding

print(df.columns)

# اختيار الأعمدة المهمة فقط بما في ذلك الأعمدة الرقمية
df = df[['gender', 'SeniorCitizen', 'tenure', 'PhoneService', 'InternetService', 'Contract', 
         'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']]

# تحويل الأعمدة الفئوية إلى متغيرات رقمية باستخدام One-Hot Encoding
df = pd.get_dummies(df, columns=['gender', 'PhoneService', 'InternetService', 'Contract', 'PaymentMethod'], drop_first=True)

# استخدام Label Encoding للأعمدة ذات قيم ثنائية مثل 'Churn'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Churn'] = le.fit_transform(df['Churn'])


In [0]:
# استخدام boxplot لاكتشاف القيم المتطرفة في الأعمدة الرقمية
import seaborn as sns
sns.boxplot(x=df['MonthlyCharges'])

# إزالة القيم المتطرفة بناءً على القيم العليا والدنيا
Q1 = df['MonthlyCharges'].quantile(0.25)
Q3 = df['MonthlyCharges'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['MonthlyCharges'] >= (Q1 - 1.5 * IQR)) & (df['MonthlyCharges'] <= (Q3 + 1.5 * IQR))]


In [0]:
from sklearn.model_selection import train_test_split

# تحديد المتغيرات المستقلة (features) والمتغير التابع (target)
X = df.drop(columns=['Churn'])  # جميع الأعمدة باستثناء العمود "Churn"
y = df['Churn']  # عمود "Churn" هو المتغير التابع

# تقسيم البيانات إلى تدريب واختبار بنسبة 80% تدريب و 20% اختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Explanation: We split the data into 80% training data and 20% testing data.
# The random_state ensures that the split is reproducible.


In [0]:
from sklearn.tree import DecisionTreeClassifier

# إنشاء نموذج شجرة القرار
dt_model = DecisionTreeClassifier(random_state=42)

# تدريب النموذج باستخدام البيانات التدريبية
dt_model.fit(X_train, y_train)

# Explanation: The DecisionTreeClassifier is trained on the training data (X_train, y_train).
# It learns how to classify the data based on the input features.


In [0]:
# التنبؤ باستخدام النموذج المدرب على بيانات الاختبار
y_pred = dt_model.predict(X_test)

# Explanation: After training the model, we use the test set (X_test) to make predictions (y_pred).
# This allows us to evaluate how well the model generalizes to unseen data.


In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# حساب الدقة (Accuracy)
accuracy = accuracy_score(y_test, y_pred)

# حساب مصفوفة الالتباس (Confusion Matrix)
conf_matrix = confusion_matrix(y_test, y_pred)

# تقرير تصنيف (Precision, Recall, F1-Score)
class_report = classification_report(y_test, y_pred)

# طباعة النتائج
print("Accuracy: ", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

# Explanation: 
# - Accuracy: the percentage of correct predictions made by the model.
# - Confusion Matrix: shows how many predictions were correct/incorrect for each class (e.g., churn vs non-churn).
# - Classification Report: provides precision, recall, and F1-score for both classes (churn and non-churn).
